# ВЫПОЛНЕНИЕ ПРОЕКТА

# Шаг 1. Импорт библиотеки Pandas, получение и "осмотр" данных.

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В глаза сразу бросается несколько вещей, а именно:
1. Отрицательные значения в колонке 'days_employed', и там же видим выброс - 340'266 отработанных дней (932 года). Можно только позавидовать. Ну, или почистить данные.
2. Одинаковые значения с разными регистрами в колонке 'education'.
3. Похожие по смыслу значения и повторяющиеся слова в столбце 'purpose'.
4. Пропуски в колонках 'days_employed' и 'total_income'.

Уже вырисовывается фронт работ. Колонку **'education'** приведём к нижнему регистру, пропуски в **'total_income'**, скорее всего, заполним средними значениями, столбец **'purpose'** будет подвержен стеммингу, а вот с **'days_employed'** придётся познакомиться получше.


# Шаг 2. Предобработка данных

В первую очередь, проверим датафрейм на наличие дубликатов, пока наше приведение всего к единому виду не увеличило их количество.

In [3]:
df.duplicated().sum()

54

Удалим дубли, пересчитаем индексы (удалив старые) и убедимся, что всё сработало

In [4]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Теперь приведём колонку **'education'** к нижнему регистру и проверим уникальные значения.

In [5]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Повторяющихся значений нет, всё ок.

Нам необходимо заполнить пропуски в столбце **'total_income'**. Поскольку уровень дохода зависит от квалификации, а она, в свою очерендь, от уровня образования, заполним пропуски средними значениями для уровня образования конкретного заёмщика.

Сгруппируем данные по столбцу **'education'** и посчитаем для каждого значения медиану по столбцу **'total_income'**. Сохраним в отдельную таблицу.

In [6]:
mean_education = df.groupby('education')['total_income'].median()
display(mean_education)

education
высшее                 175340.818855
начальное              117137.352825
неоконченное высшее    160115.398644
среднее                136478.643244
ученая степень         157259.898555
Name: total_income, dtype: float64

Теперь, назначим фнкцию, которая заполнит пропуски в соответствии с уровнем образования, по индексам из новой таблицы.

In [7]:
qualification_list = ['среднее', 'высшее', 'начальное', 'ученая степень', 'неоконченное высшее']

def fill_blank(qual_list):
    for qual in qual_list:
        df.loc[df.loc[:, 'education'] == qual, 'total_income'] = df.loc[df.loc[:, 'education'] == qual, 'total_income'].fillna(mean_education[qual])

fill_blank(qualification_list)

df['total_income'].isnull().sum()

0

Пока заполним пропуски нулями, самим столбцом, займемся чуть позже.

In [8]:
df['days_employed'] = df['days_employed'].fillna(0)

Также, у нас есть колонка **'education_id'**. Видимо, она тоже сообщает нам какую-то информацию об уровне образования заёмщика. Проверим как соотносятся эти 2 колонки друг с другом, вызовом метода **groupby()**.

In [9]:
df.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  5251
1             среднее                15188
2             неоконченное высшее      744
3             начальное                282
4             ученая степень             6
Name: education, dtype: int64

Каждому уровню образования соответствует определённая цифра, причём, с отсутствием градации. Учёная степень - 4, начальное образование - 3, высшее - 0. Сохраним эти 2 колонки в словарь, чтобы удалить лишнюю информацию из таблицы. Очистим словарь от дублей и проверим его вид.

In [10]:
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Теперь удалим столбец из датафрейма и выведем первые 5 строк таблицы, чтобы убедиться, что всё прошло нормально.

In [11]:
df.drop(columns=['education'], inplace=True)
df.head()

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Да, всё отлично.

Аналогичным образом займёмся семейным положением

In [12]:
df.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12344
1                 гражданский брак          4163
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 Не женат / не замужем     2810
Name: family_status, dtype: int64

Индексы уникальны для каждого уровня, нет повторений и ошибок. Сохраним в словарь.

In [13]:
family_dict = df[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удаляем столбец из таблицы.

In [14]:
df = df.drop(columns=['family_status'])
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Всё ОК, идём дальше. 

Проверим уникальные значения по типу дохода (точнее, занятости).

In [15]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Довольно странная ситуация... безработных и предпринимателей всего по 2 человека, в то же время не ясно, в какую более крупную категорию их можно было бы перенести... 

Безработный, это не пенсионер и не сотрудник.

Аналогично студент и работник находящийся в декрете. 

Студент, это почти что молодой и перспективный специалист, со свежими знаниями, и поэтому востребованный на рынке труда. У декретного есть работодатель, закреплённое за ним место работы, и он может вернуться на него в любой момент.

Перенесём предпринимателя к компаньонам (ведь, наверняка, он чей-то компаньон) и проверим значения ещё раз.

Остальное пока оставим как есть.

In [16]:
df['income_type'] = df['income_type'].replace('предприниматель', value='компаньон')
df['income_type'].value_counts()

сотрудник      11091
компаньон       5082
пенсионер       3837
госслужащий     1457
безработный        2
в декрете          1
студент            1
Name: income_type, dtype: int64

Отлично, идём дальше.

Что там было по лемме/стемме?

In [17]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Тут мы явно видим несколько категорий. Свадбы, Автомобили и Образование, всё остальное относится к недвижимости.

Лемматизировать не будем, так как pymystem3 работает довольно медленно. Решим вопрос через стемминг.

In [18]:
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

print(russian_stemmer.stem('образование'))
print(russian_stemmer.stem('свадьба'))
print(russian_stemmer.stem('автомобиль'))

образован
свадьб
автомобил


Теперь, напишем функцию одной строки, которая будет проверять наличие стеммы от ключевых слов в ячейках столбца **'purpose'**. При обнаружении, функция возвращает ключевое слово.

Применим функцию к самому столбцу **'purpose'** и перепишем его значения.

In [19]:
keys = ['автомобиль', 'свадьба', 'образование']

def purpose_edit(row):
    for key in keys: 
        if russian_stemmer.stem(key) in row['purpose']:
            return key


df['purpose'] = df.apply(purpose_edit, axis=1)
df['purpose'].value_counts()

автомобиль     4308
образование    4014
свадьба        2335
Name: purpose, dtype: int64

Потеряли всю недвижимость. Её значения остались пустыми. Проверим это.

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 10 columns):
children            21471 non-null int64
days_employed       21471 non-null float64
dob_years           21471 non-null int64
education_id        21471 non-null int64
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float64
purpose             10657 non-null object
dtypes: float64(2), int64(5), object(3)
memory usage: 1.6+ MB


Заполним пропуски в столбце **'purpose'** словом **недвижимость**

In [21]:
df['purpose'] = df['purpose'].fillna('недвижимость')
df['purpose'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose, dtype: int64

Отлично. Теперь проверим уникальные поля столбца **'gender'**.

In [22]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Указано непонятное значение, рассмотрим эту строку.

In [23]:
df.loc[df.loc[:, 'gender'] == 'XNA']

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
10690,0,-2358.600502,24,2,1,XNA,компаньон,0,203905.157261,недвижимость



Предположу, что это лицо мужского пола, судя по 6-ти летнему опыту работы уже в 24 года, и коммерческой активности.
В то же время, большая часть заёмщиков в выборке - женского пола, следуя этой логике надо заполнить значением 'F'.

Осмелюсь довериться личному мнению и заполню значение буквой 'M'.

In [24]:
df['gender'] = df['gender'].replace('XNA', value='M')
df.loc[10690]

children                       0
days_employed            -2358.6
dob_years                     24
education_id                   2
family_status_id               1
gender                         M
income_type            компаньон
debt                           0
total_income              203905
purpose             недвижимость
Name: 10690, dtype: object

Успешно.

А вообще, лучше изменить значения этих ячеек на более понятные нам мужской и женский пол.

In [25]:
df['gender'] = df['gender'].replace('F', value='женский')
df['gender'] = df['gender'].replace('M', value='мужской')
df['gender'].value_counts()

женский    14189
мужской     7282
Name: gender, dtype: int64

Сработало. 

Теперь перейдём к числовым данным.

In [26]:
df['debt'].value_counts()

0    19730
1     1741
Name: debt, dtype: int64

Всё просто и ясно. **1** - были пролемы, **0** - не было.

Рассмотрим данные о возрасте заёмщиков

In [27]:
df['dob_years'].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
56    484
52    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Всё бы неплохо, но 101 человек с возрастом 0.

Заполним эти нули средним значением. А то удалять 101 запись, как-то не хочется.

Сравним среднее и медианное значения. Предположу, что они будут близки по значению.

In [28]:
mean_years = df['dob_years'].mean()
median_years = df['dob_years'].median()
print(mean_years, median_years)

43.279074099948765 42.0


Так и оказалось. Голосую за медиану.

In [29]:
df['dob_years'] = df['dob_years'].replace(0, value=int(median_years))
df['dob_years'].value_counts()

42    697
35    616
40    607
41    606
34    601
38    597
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Теперь столбец **'children'**

In [30]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Заменинм -1 на 1. Тут всё очевидно.

In [31]:
df['children'] = df['children'].replace(-1, value=1)
df['children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

А вот 20 детей, это серьёзно. Либо 2 либо 0. Предполагаю, что 2. Рассмотрим повнимательнее эти строки.

In [32]:
df.loc[df.loc[:, 'children'] == 20].head(20)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,1,0,мужской,компаньон,0,145334.865002,недвижимость
720,20,-855.595512,44,1,0,женский,компаньон,0,112998.738649,недвижимость
1074,20,-3310.411598,56,1,0,женский,сотрудник,1,229518.537004,образование
2510,20,-2714.161249,59,0,2,женский,сотрудник,0,264474.835577,недвижимость
2940,20,-2161.591519,42,1,0,женский,сотрудник,0,199739.941398,автомобиль
3301,20,0.000000,35,1,4,женский,госслужащий,0,136478.643244,образование
3395,20,0.000000,56,0,0,женский,компаньон,0,175340.818855,образование
3670,20,-913.161503,23,1,4,женский,сотрудник,0,101255.492076,автомобиль
3696,20,-2907.910616,40,1,1,мужской,сотрудник,0,115380.694664,автомобиль
3734,20,-805.044438,26,0,4,мужской,сотрудник,0,137200.646181,недвижимость


Люди разные. Встречаются и не женатые, и совсем молодые. Сколько из них имели проблемы с возвратом долга?

In [33]:
df.loc[(df['children'] == 20) & (df['debt'] == 1)]['debt'].count()

8

8 из 76. Это больше чем в среднем по выборке (8,8 % против 10,52 %). То есть данные значимые. 

Заполним их медианой и пойдём дальше.

In [34]:
df['children'].median()

0.0

In [35]:
df['children'] = df['children'].replace(20, value=0)
df['children'].value_counts()

0    14183
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Нам это удалось. Теперь данные о количестве отработанных дней. Смотреть все уникальные значения мы не будем, так как вариантов столько же сколько строк, и это теряет смысл.

Многие из этих ячеек имеют отрицательные значения. Исправим это и проверим результат

In [36]:
df['days_employed'] = df['days_employed'].abs()
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,женский,сотрудник,0,253875.639453,недвижимость
1,1,4024.803754,36,1,0,женский,сотрудник,0,112080.014102,автомобиль
2,0,5623.422610,33,1,0,мужской,сотрудник,0,145885.952297,недвижимость
3,3,4124.747207,32,1,0,мужской,сотрудник,0,267628.550329,образование
4,0,340266.072047,53,1,1,женский,пенсионер,0,158616.077870,свадьба
5,0,926.185831,27,0,1,мужской,компаньон,0,255763.565419,недвижимость
6,0,2879.202052,43,0,0,женский,компаньон,0,240525.971920,недвижимость
7,0,152.779569,50,1,0,мужской,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,0,1,женский,сотрудник,0,95856.832424,свадьба
9,0,2188.756445,41,1,0,мужской,сотрудник,0,144425.938277,недвижимость


Вот и славно. 

Далее, преобразуем все вещественные числа в целочисленные, для удобства восприятия.

In [37]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,женский,сотрудник,0,253875,недвижимость
1,1,4024,36,1,0,женский,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,мужской,сотрудник,0,145885,недвижимость
3,3,4124,32,1,0,мужской,сотрудник,0,267628,образование
4,0,340266,53,1,1,женский,пенсионер,0,158616,свадьба
5,0,926,27,0,1,мужской,компаньон,0,255763,недвижимость
6,0,2879,43,0,0,женский,компаньон,0,240525,недвижимость
7,0,152,50,1,0,мужской,сотрудник,0,135823,образование
8,2,6929,35,0,1,женский,сотрудник,0,95856,свадьба
9,0,2188,41,1,0,мужской,сотрудник,0,144425,недвижимость


Снова на глаза попался артефакт с 1000-летним человеком. Может он такой не один?

Отсортируем таблицу по убыванию и посмотрим несколько первых строк.

In [38]:
df['days_employed'].sort_values(ascending=False).head(10)

6950     401755
9997     401715
7660     401675
2156     401674
7790     401663
4695     401635
13402    401619
17789    401614
10979    401591
8364     401590
Name: days_employed, dtype: int64

Понятно... но сколько же их?

Запросим побольше (после нескольких проб, нащупал нужное число).

In [39]:
df['days_employed'].sort_values(ascending=False).head(3448)

6950     401755
9997     401715
7660     401675
2156     401674
7790     401663
          ...  
9321     328734
20395    328728
16307     18388
4297      17615
7325      16593
Name: days_employed, Length: 3448, dtype: int64

Почти 3,5 тысячи значений. Либо это эльфы, либо ошибка в данных.

Создадим новую таблицу, отобрав из старой некорректные записи. Минимальное значение **328'728**, возьмём за ориентир 300'000.

Посмотрим на данные повнимательнее.

In [40]:
empl_err = df.loc[df.loc[:, 'days_employed'] > 300000]
empl_err.head(20)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266,53,1,1,женский,пенсионер,0,158616,свадьба
18,0,400281,53,1,2,женский,пенсионер,0,56823,автомобиль
24,1,338551,57,1,4,женский,пенсионер,0,290547,недвижимость
25,0,363548,67,1,0,мужской,пенсионер,0,55112,недвижимость
30,1,335581,62,1,0,женский,пенсионер,0,171456,недвижимость
35,0,394021,68,1,1,мужской,пенсионер,0,77805,свадьба
50,0,353731,63,1,0,женский,пенсионер,0,92342,автомобиль
56,0,370145,64,1,2,женский,пенсионер,0,149141,образование
71,0,338113,62,1,0,женский,пенсионер,0,43929,автомобиль
78,0,359722,61,0,0,мужской,пенсионер,0,175127,автомобиль


Пенсионеры... Логично, при таком-то стаже...

Сгруппируем таблицу по 'income_type' и подсчитаем, кого и сколько.

In [41]:
empl_err.groupby('income_type').count()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2
пенсионер,3443,3443,3443,3443,3443,3443,3443,3443,3443


Да, так и есть. Но откуда у безработных такой огромный стаж? Их всего двое. Рассмотрим данные по всем безработным в исходной таблице.

In [42]:
df.loc[df.loc[:, 'income_type'] == 'безработный'].head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
3132,1,337524,31,1,0,мужской,безработный,1,59956,недвижимость
14775,0,395302,45,0,1,женский,безработный,0,202722,недвижимость


А их всего двое и было. Странная картина. Работы нет, но есть огромный стаж и внушительные доходы... Похоже на артефакт. 

Удаляю 2 строки и пересчитываю индексы, чтобы зря не ломать голову. 

Остаёмся без безработных как группы, но их было слишком мало для исследовния, и по ним всё, в принципе, тривиально. В нашей выборке 50/50 ))) Если конечно данные столбца 'debt' не искажены. 

Выведем один из фрагментов таблицы, для проверки сброса индексов.

In [43]:
df = df.drop(index=[3132, 14775])
df = df.reset_index(drop=True)
df[3130:3134]

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
3130,0,328,50,1,0,мужской,сотрудник,0,74296,недвижимость
3131,1,1490,51,1,0,мужской,сотрудник,0,243545,недвижимость
3132,0,397644,60,1,4,женский,пенсионер,0,106106,недвижимость
3133,0,2967,63,1,0,мужской,госслужащий,0,56923,автомобиль


Сработало. Проверим безработных ещё раз

In [44]:
df.loc[df.loc[:, 'income_type'] == 'безработный'].head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose


Отлично. 

Теперь категоризируем данные по количественным показателям.

Напишем функцию одной строки, с помощью которой сможем обобщить сведения о доходах заёмщиков. 

Предлагаю 4 категории, с доходом: 

- менее 50 тысяч рублей

- от 50 до 100 тысяч рублей

- от 100 до 150 тысяч рублей

- более 150 тысяч рублей

Перезапишем с помощью этой функции значения самого же столбца 'total_income'

In [45]:
def income_cat(row):
    if row['total_income'] > 150000:
        return 'более 150 тыс.'
    elif 100000 <= row['total_income'] <= 150000:
        return '100 - 150 тыс.'
    elif 50000 <= row['total_income'] <= 100000:
        return '50 - 100 тыс.'
    elif row['total_income'] < 50000:
        return 'менее 50 тыс.'
        
df['total_income'] = df.apply(income_cat, axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,женский,сотрудник,0,более 150 тыс.,недвижимость
1,1,4024,36,1,0,женский,сотрудник,0,100 - 150 тыс.,автомобиль
2,0,5623,33,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость
3,3,4124,32,1,0,мужской,сотрудник,0,более 150 тыс.,образование
4,0,340266,53,1,1,женский,пенсионер,0,более 150 тыс.,свадьба


In [46]:
df['total_income'].value_counts()

более 150 тыс.    9787
100 - 150 тыс.    7220
50 - 100 тыс.     4090
менее 50 тыс.      372
Name: total_income, dtype: int64

Есть, все хорошо.

Теперь, опять с помощью функции одной строки, посчитаем сколько лет трудового стажа у наших заёмщиков.

А потом отнимем от эту цифру от их возраста и узнаем сколько им было лет, когда они начали трудиться. Данные запишем в столбцы 'work_years' и 'work_start' соответственно.

А потом посмотрим есть ли среди наших заёмщиков те, кто трудится с пелёнок. С 14 лет разрешены некоторые виды труда с согласия родителей, в свободное от учебы время. Посчитаем количество тех, кто начал трудиться раньше.

In [47]:
def work_years(row):
    return row['days_employed']/365


df['work_years'] = df.apply(work_years, axis=1)
df['work_start'] = df['dob_years'] - df['work_years']
df.loc[(df.loc[:, 'work_start'] > 0) & (df.loc[:, 'work_start'] < 14)].count()

children            18
days_employed       18
dob_years           18
education_id        18
family_status_id    18
gender              18
income_type         18
debt                18
total_income        18
purpose             18
work_years          18
work_start          18
dtype: int64

Нашлись 18 человек. Возможно ошибка в данных. Рассмотрим их поближе.

In [48]:
df.loc[(df.loc[:, 'work_start'] > 0) & (df.loc[:, 'work_start'] < 14)].head(18)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,work_years,work_start
397,0,12506,46,1,0,женский,сотрудник,0,более 150 тыс.,недвижимость,34.263014,11.736986
2082,0,10689,42,1,0,женский,сотрудник,0,более 150 тыс.,недвижимость,29.284932,12.715068
2492,0,13724,50,0,0,женский,сотрудник,0,50 - 100 тыс.,недвижимость,37.600000,12.400000
3415,1,5673,29,1,0,мужской,сотрудник,0,более 150 тыс.,образование,15.542466,13.457534
3955,0,12111,47,1,0,женский,компаньон,0,100 - 150 тыс.,недвижимость,33.180822,13.819178
4296,0,17615,61,1,0,женский,компаньон,0,100 - 150 тыс.,недвижимость,48.260274,12.739726
5576,0,15079,55,1,0,женский,госслужащий,0,более 150 тыс.,недвижимость,41.312329,13.687671
5703,0,13210,47,1,0,женский,сотрудник,0,100 - 150 тыс.,недвижимость,36.191781,10.808219
5954,0,10939,42,1,0,женский,сотрудник,0,50 - 100 тыс.,автомобиль,29.969863,12.030137
8728,0,14240,53,1,0,мужской,компаньон,0,более 150 тыс.,недвижимость,39.013699,13.986301


Что неправильно в этих записях? Возраст или стаж? В целом, люди из списка представляют большинство, почти по всем признакам:

- проблем с долгами не было,
- цель кредита - недвижимость,
- пол женский,
- детей нет,
- образование среднее,
- состоят в браке.

Считаю, что эти строки можно удалить без ущерба для результатов исследования.

Селаем выборку из таблицы тех заёмщиков, которые начали трудиться в 14 и старше, или в ноль и меньше, чтобы не потерять тех у кого в стаже проставлено 0, или ещё не обработанных эльфов.

In [49]:
df = df[(df['work_start'] >= 14) | (df['work_start'] <= 0)]
df.loc[(df.loc[:, 'work_start'] > 0) & (df.loc[:, 'work_start'] < 14)].head(18)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,work_years,work_start


Данные удалены успешно.

Через функцию одной строки категоризируем трудовой стаж, по группам:

- менее года,
- 1-5 лет,
- 5-10 лет,
- 10-20 лет,
- неизвестно (эльфы и пропуски, заполненные нулями)

In [50]:
def work_exp(row):
     if 0 < row['work_years'] < 1:
        return 'менее 1 года'
     elif 1 <= row['work_years'] <= 5:
        return '1 - 5 лет'
     elif 5 <= row['work_years'] <= 10:
        return '5 - 10 лет'
     elif 10 <= row['work_years'] <= 20:
        return '10 - 20 лет'
     elif 20 < row['work_years'] < 100:
        return 'более 20 лет'
     else:
        return 'неизвестно'
    
df['work_years'] = df.apply(work_exp, axis=1)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,work_years,work_start
0,1,8437,42,0,0,женский,сотрудник,0,более 150 тыс.,недвижимость,более 20 лет,18.884932
1,1,4024,36,1,0,женский,сотрудник,0,100 - 150 тыс.,автомобиль,10 - 20 лет,24.975342
2,0,5623,33,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость,10 - 20 лет,17.594521
3,3,4124,32,1,0,мужской,сотрудник,0,более 150 тыс.,образование,10 - 20 лет,20.701370
4,0,340266,53,1,1,женский,пенсионер,0,более 150 тыс.,свадьба,неизвестно,-879.235616
5,0,926,27,0,1,мужской,компаньон,0,более 150 тыс.,недвижимость,1 - 5 лет,24.463014
6,0,2879,43,0,0,женский,компаньон,0,более 150 тыс.,недвижимость,5 - 10 лет,35.112329
7,0,152,50,1,0,мужской,сотрудник,0,100 - 150 тыс.,образование,менее 1 года,49.583562
8,2,6929,35,0,1,женский,сотрудник,0,50 - 100 тыс.,свадьба,10 - 20 лет,16.016438
9,0,2188,41,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость,5 - 10 лет,35.005479


Всё готово. избавимся от лишних столбцов

In [51]:
df.drop(columns=['work_start', 'days_employed'], inplace=True)
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,work_years
0,1,42,0,0,женский,сотрудник,0,более 150 тыс.,недвижимость,более 20 лет
1,1,36,1,0,женский,сотрудник,0,100 - 150 тыс.,автомобиль,10 - 20 лет
2,0,33,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость,10 - 20 лет
3,3,32,1,0,мужской,сотрудник,0,более 150 тыс.,образование,10 - 20 лет
4,0,53,1,1,женский,пенсионер,0,более 150 тыс.,свадьба,неизвестно


Теперь категоризируем возраст, аналогичной функцией одной строки.

Категории:

- младше 25 лет,
- 25-40 лет,
- 40-60 лет,
- старше 60 лет.

Выведем первые 5 строк и проверим результат.

In [52]:
def years_cat(row):
    if row['dob_years'] > 60:
        return 'старше 60 лет'
    elif 40 <= row['dob_years'] <= 60:
        return '40 - 60 лет'
    elif 25 <= row['dob_years'] <= 40:
        return '25 - 40 лет'
    elif row['dob_years'] < 25:
        return 'младше 25 лет'
        
df['dob_years'] = df.apply(years_cat, axis=1)
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,work_years
0,1,40 - 60 лет,0,0,женский,сотрудник,0,более 150 тыс.,недвижимость,более 20 лет
1,1,25 - 40 лет,1,0,женский,сотрудник,0,100 - 150 тыс.,автомобиль,10 - 20 лет
2,0,25 - 40 лет,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость,10 - 20 лет
3,3,25 - 40 лет,1,0,мужской,сотрудник,0,более 150 тыс.,образование,10 - 20 лет
4,0,40 - 60 лет,1,1,женский,пенсионер,0,более 150 тыс.,свадьба,неизвестно


Теперь заменим названия заголовков таблицы, на более подходящие.

In [53]:
df.rename(columns={'dob_years':'age', 'income_type':'employ_type', 'total_income':'mounth_income', 'work_years':'experience'}, inplace=True)
df.head()

,children,age,education_id,family_status_id,gender,employ_type,debt,mounth_income,purpose,experience
0,1,40 - 60 лет,0,0,женский,сотрудник,0,более 150 тыс.,недвижимость,более 20 лет
1,1,25 - 40 лет,1,0,женский,сотрудник,0,100 - 150 тыс.,автомобиль,10 - 20 лет
2,0,25 - 40 лет,1,0,мужской,сотрудник,0,100 - 150 тыс.,недвижимость,10 - 20 лет
3,3,25 - 40 лет,1,0,мужской,сотрудник,0,более 150 тыс.,образование,10 - 20 лет
4,0,40 - 60 лет,1,1,женский,пенсионер,0,более 150 тыс.,свадьба,неизвестно


Ещё раз проверим общую информацию о датасете.

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21451 entries, 0 to 21468
Data columns (total 10 columns):
children            21451 non-null int64
age                 21451 non-null object
education_id        21451 non-null int64
family_status_id    21451 non-null int64
gender              21451 non-null object
employ_type         21451 non-null object
debt                21451 non-null int64
mounth_income       21451 non-null object
purpose             21451 non-null object
experience          21451 non-null object
dtypes: int64(4), object(6)
memory usage: 1.8+ MB


Потеряли лишь около 0.3% исходных данных. Отлично.

Данные готовы для анализа. Я надеюсь.

# Шаг 3. Ответы на вопросы

Для того, чтобы ответить на поставленные вопросы, выведем данные спомощью сводных таблиц по интересующим нас признакам.

## Наличие детей.

Специально не стал категоризировать эти данные, чтобы проверить есть ли зависимость между количсетвом детей и возвратом кредита в срок.

In [55]:
child_pivot = df.pivot_table(index=['children'], columns='debt', values='education_id', aggfunc='count')
child_pivot.head(10)

debt,0,1
children,,
0,13095.0,1071.0
1,4409.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Добавим ещё один столбец, рассчитав процент невозврата от общего числа займов.

In [56]:
child_pivot['percent'] = child_pivot[1]/(child_pivot[1] + child_pivot[0]) * 100
child_pivot.head(10)

debt,0,1,percent
children,,,
0,13095.0,1071.0,7.560356
1,4409.0,444.0,9.148980
2,1858.0,194.0,9.454191
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


На вопрос, влияет ли наличие детей на возврат кредита в срок, ответ **ДА**. У людей с пятью детьми, действительно, ни разу не было проблем с возвратом, возможно от того, что по ним было мало наблюдений. У остальных имевших детей, процент проблем был выше. Возможно потому, что детям сложнее в чём-то отказать, и затрат, в принципе, больше. В то же время, зависимости между количеством детей и возвратом кредита не обнаружено.

## Семейное положение

Помню, что мы выводили семейное положение в словарь, а в исходной таблице оставили лишь id. Применим метод **merge()**, и выведем сводную таблицу на экран.

In [57]:
family_status_pivot = df.merge(family_dict, how='left', left_on='family_status_id', right_on='family_status_id').pivot_table(index=['family_status'], columns='debt', values='education_id', aggfunc='count')
family_status_pivot.head(10)

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3773,388
женат / замужем,11396,930


"Смёрджить" удалось.

Аналогично предыдущему случаю, рассчитаем процент невозврата.

In [58]:
family_status_pivot['percent'] = family_status_pivot[1]/(family_status_pivot[1] + family_status_pivot[0]) * 100
family_status_pivot.head(10)

debt,0,1,percent
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3773,388,9.324682
женат / замужем,11396,930,7.545027


На вопрос есть ли зависимость между семейным положением и возвратом кредита в срок, ответ **ДА**. У холостых и людей проживающих в гражданском браке, процент проблем был самый высокий. У женатых проблем с возвратом меньше. У разведённых и вдовствующих заёмщиков, проблем оказалось меньше всех. Возможно, потому, что меньше затрат чем у женатых, а уровень ответственности остался высоким.

## Уровень дохода.

In [59]:
income_pivot = df.pivot_table(index=['mounth_income'], columns='debt', values='education_id', aggfunc='count')
income_pivot['percent'] = income_pivot[1]/(income_pivot[1] + income_pivot[0]) * 100
income_pivot.head(10)

debt,0,1,percent
mounth_income,,,
100 - 150 тыс.,6581,632,8.761958
50 - 100 тыс.,3757,330,8.074382
более 150 тыс.,9024,755,7.720626
менее 50 тыс.,349,23,6.182796


На вопрос есть ли зависимость между уровнем дохода и возвратом кредита в срок, ответ, скорее, **НЕТ**. Лучше всех отдают долги люди, с доходом менее 50 тысяч. Люди с доходом от 50 до 100 тысяч, лучше отдают долги чем люди, с доходом от 100 до 150 тысяч, при этом, люди с доходом более 150 тысяч, отдают долги лучше, чем люди в двух последних категориях. Вероятно, чем больше доходы, тем больше расходы и аппетиты.

## Цели кредита.

In [60]:
purpose_pivot = df.pivot_table(index=['purpose'], columns='debt', values='education_id', aggfunc='count')
purpose_pivot['percent'] = purpose_pivot[1]/(purpose_pivot[1] + purpose_pivot[0]) * 100
purpose_pivot.head(10)

debt,0,1,percent
purpose,,,
автомобиль,3904,403,9.356861
недвижимость,10018,781,7.232151
образование,3641,370,9.224632
свадьба,2148,186,7.969152


Отвечая на вопрос как разные цели влияют на возврат кредита в срок, можно сказать, что лучше всего возвращают кредит за недвижимость, хуже всего за автомобиль и почти так же за образование. Автомобили склонны ломаться, попадать в аварии, и без них не трудно обойтись. Образование не всегда можно окупить хорошей работой после обучения. А вот жильё, вещь необходимая.

# Шаг 4. Общий вывод

Исходных данные не были единообразными. Наверняка, разные менеджеры банка по-разному заполняли данные о заёмщиках и целях кредита. Возможно, ошибки происходили при выгрузке данных разработчиками.
Ошибки в данных с которыми я столкнулся:
- дубликаты,
- пропуски в столбцах **'total_income'** и **'days_employed'**,
- значения **0** в столбце **'dob_years'**,
- отрицательные значения в столбцах **'days_employed'** и **'children'**,
- артефакты (выбросы) в столбцах **'children'** и **'days_employed'**,
- непонятное значение **'XNA'** в столбце **'gender'**,
- тип данных **float** в столбцах **'total_income'** и **'days_employed'**,
- разнообразным образом записанные, но одинаковые цели кредита в столбце **'purpose'**,
- безработные с внушительным доходом,
- не совсем корректные наименования столбцов.

Для приведения данных таблицы к единому виду, для удобства работы с ними и составления выводов, мне пришлось:
- удалить дубликаты,
- заполнить пропуски в столбце **'total_income'** (методом **fillna()**), медианой в зависимости от уровня дохода,
- заменить нулевой возраст медианным в столбце **'dob_years'** (метод **replace()**),
- заполнить пропуски столбца **'days_employed'** сначала нулём (**fillna()**), чтобы не нарушать тип данных **int** в столбце, а при категоризации словом **"неизвестно"**,
- заменить отрицательные значения на положительные в столбцах **'children'** (через **replace()**) и **'days_employed'** (методом **abs()**),
- заменить выбросы в столбце **'children'** на медианные значения, заменить некорректные значения (выбросы) столбца **'days_employed'**, словом **неизвестно**,
- заменить значение **'XNA'** в столбце **'gender'** на значение **M**, исходя из других наболюдений по этому заёмщику,
- удалить записи о безработных, так как их было всего двое и у них был записан огромный стаж и хороший доход (метод **drop()** и пересчёт индексов через **reset_index(drop=True)**),
- удалить записи о лицах, начавших трудиться в 10-13 лет (ошибка либо в **'days_employed'**, либо в **'dob_years'**),
- провести стемминг столбца **'purpose'**, для приведения целей кредита к 4-м основным значениям,
- перенести **предпринимателей** в категорию **компаньоны**, так как первых было всего 2 человека,
- убрать из таблицы лишние столбцы, создав отдельные таблицы-словари, оставив в таблице только числовые столбцы **id** для семейного положения и уровня образования,
- категоризировать данные по доходу, возрасту и трудовому стажу, чтобы не путаться в многообразии значений,
- создать сводные таблицы по целевым признакам для соответствующих выводов, и добавить в них новую колонку с расчётом доли отказа.

После выполнения всех этих действий, мне удалось легко найти нужные цифры и ответить на все поставленные вопросы.